# ⚡ Batch Processing - Scale to 1000s of URLs

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vzucher/brightdata-sdk-python/blob/master/notebooks/05_batch_processing.ipynb)

Learn how to efficiently scrape thousands of URLs with progress tracking, error handling, and cost management.

## What You'll Learn
1. Progress bars with tqdm
2. Error handling at scale
3. Cost tracking and budgets
4. Caching for development
5. Parallel processing
6. Resume interrupted jobs

---


In [ ]:
%pip install brightdata-sdk pandas tqdm joblib -q

import pandas as pd
from tqdm.auto import tqdm
import joblib
from brightdata import BrightDataClient

API_TOKEN = "your_api_token_here"
client = BrightDataClient(token=API_TOKEN)
print("✅ Ready for batch processing!")


## 📊 1. Progress Bars with tqdm

Always show progress when scraping multiple URLs:


In [ ]:
# Generate sample URLs
urls = [
    f"https://www.amazon.com/dp/B0{i:08d}" 
    for i in range(10)  # Start with 10 for demo
]

results = []
total_cost = 0

# Scrape with progress bar
for url in tqdm(urls, desc="Scraping Amazon products"):
    try:
        result = client.scrape.amazon.products(url=url)
        
        if result.success:
            results.append({
                'url': url,
                'title': result.data.get('title', 'N/A'),
                'price': result.data.get('final_price', 'N/A'),
                'cost': result.cost,
                'status': 'success'
            })
            total_cost += result.cost
        else:
            results.append({
                'url': url,
                'error': result.error,
                'cost': 0,
                'status': 'failed'
            })
    except Exception as e:
        results.append({
            'url': url,
            'error': str(e),
            'cost': 0,
            'status': 'error'
        })

df = pd.DataFrame(results)
print(f"\n✅ Processed {len(df)} URLs")
print(f"💰 Total cost: ${total_cost:.4f}")
print(f"✅ Success: {(df['status'] == 'success').sum()}")
print(f"❌ Failed: {(df['status'] != 'success').sum()}")


## 💰 2. Cost Management and Budgets

Stop scraping when you reach a budget limit:


In [ ]:
# Set a budget
BUDGET_LIMIT = 1.00  # $1.00 budget
total_cost = 0
results_with_budget = []

print(f"💰 Budget: ${BUDGET_LIMIT:.2f}")

for url in tqdm(urls, desc="Scraping with budget"):
    # Check budget
    if total_cost >= BUDGET_LIMIT:
        print(f"\n⚠️  Budget limit reached! Stopping at ${total_cost:.4f}")
        break
    
    try:
        result = client.scrape.amazon.products(url=url)
        total_cost += result.cost
        
        if result.success:
            results_with_budget.append({
                'url': url,
                'cost': result.cost,
                'cumulative_cost': total_cost
            })
            
        # Warn when approaching limit
        if total_cost > BUDGET_LIMIT * 0.8:
            print(f"\n⚠️  80% of budget used: ${total_cost:.4f}")
            
    except Exception as e:
        print(f"\n❌ Error: {e}")
        continue

print(f"\n✅ Scraped {len(results_with_budget)} URLs")
print(f"💰 Final cost: ${total_cost:.4f}")
print(f"📊 Budget used: {100 * total_cost / BUDGET_LIMIT:.1f}%")


In [ ]:
# Setup cache
memory = joblib.Memory('.cache', verbose=0)

@memory.cache
def scrape_cached(url):
    """Cached scraping - only scrapes once per URL."""
    result = client.scrape.amazon.products(url=url)
    return result.to_dict()

# First run - hits API
print("First run (hits API):")
result1 = scrape_cached(urls[0])
print(f"✅ Scraped: {urls[0][:50]}")

# Second run - uses cache (free!)
print("\nSecond run (uses cache):")
result2 = scrape_cached(urls[0])
print(f"✅ From cache: {urls[0][:50]}")

print("\n💡 Tip: Delete .cache folder to refresh cached data")


## 🔄 4. Resume Interrupted Jobs

Save progress and resume if interrupted:


In [ ]:
import os

CHECKPOINT_FILE = 'scraping_progress.csv'

# Load previous progress if exists
if os.path.exists(CHECKPOINT_FILE):
    progress_df = pd.read_csv(CHECKPOINT_FILE)
    completed_urls = set(progress_df['url'].tolist())
    print(f"📂 Resuming: {len(completed_urls)} URLs already completed")
else:
    progress_df = pd.DataFrame()
    completed_urls = set()
    print("🆕 Starting fresh")

# Process remaining URLs
remaining_urls = [url for url in urls if url not in completed_urls]
print(f"📋 {len(remaining_urls)} URLs to process")

for url in tqdm(remaining_urls, desc="Scraping"):
    try:
        result = client.scrape.amazon.products(url=url)
        
        # Save progress after each successful scrape
        if result.success:
            new_row = pd.DataFrame([{
                'url': url,
                'title': result.data.get('title'),
                'cost': result.cost,
                'timestamp': pd.Timestamp.now()
            }])
            progress_df = pd.concat([progress_df, new_row], ignore_index=True)
            progress_df.to_csv(CHECKPOINT_FILE, index=False)
            
    except KeyboardInterrupt:
        print(f"\n⚠️  Interrupted! Progress saved to {CHECKPOINT_FILE}")
        print(f"✅ Completed: {len(progress_df)} URLs")
        break
    except Exception as e:
        print(f"\n❌ Error on {url}: {e}")
        continue

print(f"\n✅ Total completed: {len(progress_df)} URLs")


## 📊 5. Batch Results Analysis


In [ ]:
# Analyze batch results
if len(df) > 0:
    print("📊 Batch Processing Summary:")
    print(f"  Total URLs: {len(df)}")
    print(f"  Success rate: {100 * (df['status'] == 'success').sum() / len(df):.1f}%")
    print(f"  Total cost: ${df['cost'].sum():.4f}")
    print(f"  Avg cost per URL: ${df['cost'].mean():.4f}")
    print(f"  Avg cost per success: ${df[df['status'] == 'success']['cost'].mean():.4f}")
    
    # Export final results
    df.to_csv('batch_results_final.csv', index=False)
    print(f"\n✅ Exported to batch_results_final.csv")


## 💡 Pro Tips for Large-Scale Scraping

### 1. Batch Size Optimization
```python
# Process in batches of 100
batch_size = 100
for i in range(0, len(urls), batch_size):
    batch = urls[i:i+batch_size]
    # Process batch
```

### 2. Rate Limiting (Built-in!)
The SDK automatically handles rate limiting - no need to add delays!

### 3. Error Recovery
```python
max_retries = 3
for retry in range(max_retries):
    try:
        result = client.scrape.amazon.products(url=url)
        break
    except Exception as e:
        if retry == max_retries - 1:
            print(f"Failed after {max_retries} retries")
```

### 4. Memory Management
```python
# For very large batches, write to CSV incrementally
with open('results.csv', 'a') as f:
    for url in urls:
        result = scrape(url)
        result_df = pd.DataFrame([result])
        result_df.to_csv(f, header=f.tell()==0, index=False)
```

---

## ✅ Summary

You learned:
- ✅ Progress tracking with tqdm
- ✅ Budget management and cost tracking
- ✅ Caching for development
- ✅ Resuming interrupted jobs
- ✅ Large-scale scraping best practices

## 🎉 Congratulations!

You've completed all notebooks! You now know how to:
1. ✅ Get started quickly
2. ✅ Work with pandas DataFrames
3. ✅ Scrape Amazon products
4. ✅ Analyze LinkedIn jobs
5. ✅ Scale to thousands of URLs

## 📚 Next Steps

- [SDK Documentation](https://github.com/vzucher/brightdata-sdk-python)
- [API Reference](https://github.com/vzucher/brightdata-sdk-python/tree/master/docs)
- [More Examples](https://github.com/vzucher/brightdata-sdk-python/tree/master/examples)

**Happy Large-Scale Scraping! ⚡**
